# Dask Naive Auto-Chunking

This experiment aims to evaluate the behavior of Dask's auto-chunking feature.
On this notebook you will find:
- The problem statement
- The data collection for the experiment
- The evaluation of the experiment results.

## Problem statement

Dask is a powerful tool for parallelizing computations in Python, especially when dealing with large datasets. However, memory management can become a challenge when working with operations that require significant resources. Unlike Python's lazy garbage collector, Dask manages memory by chunking data into smaller blocks for processing. The way Dask chunks data can affect both performance and memory usage.

This experiment aims to evaluate the impact of Dask's auto-chunking feature versus manually defined static chunk sizes. Specifically, we will analyze how Dask handles memory-intensive operations and compare the performance under the following conditions:

- When Dask auto-chunks data, allowing the system to manage chunk sizes dynamically.
- When a fixed static chunk size is manually defined.
- When operators consume excessive memory, testing the limits where Dask fails to process due to memory exhaustion.

We will observe the performance trade-offs and failure points to identify the most effective chunking strategy for memory-intensive Dask operations.

## Data Collection

In this section, we will outline the steps needed to collect the necessary data for our experiment.
The process is organized into the following steps:

1. **Setup Environment:**
  - Set up the environment with proper env variables and global constants to use during the experiment.

2. **Setup dependencies:**
  - Set up the virtual environment running this notebook with the required dependencies.

3. **Setup the output directory:**
  - On this step we will setup the output directory in which we will save the experiment results.

4. **Generate synthetic seismic data:**
  - Generate synthetic seismic data for a given shape.

5. **Collect data for each operator:**
  - Apply each operator to the synthetic data using both Dask auto-chunking, as well as a static chunk size

After completing these steps, we will have the performance data from Dask to compare the results

### Setup Environment

During the environment setup, we need to:
- Proper configure `PYTHONPATH`
- Setup dependencies

Below, we're configuring the `PYTHONPATH` to allow using the tools we've coded for the experiments

In [1]:
import os
import sys

seismic_path = os.path.abspath('../tools/seismic')
traceq_path = os.path.abspath('../tools/traceq')

if seismic_path not in sys.path:
    sys.path.append(seismic_path)

if traceq_path not in sys.path:
    sys.path.append(traceq_path)

print(sys.path)

['/home/delucca/.pyenv/versions/3.10.14/lib/python310.zip', '/home/delucca/.pyenv/versions/3.10.14/lib/python3.10', '/home/delucca/.pyenv/versions/3.10.14/lib/python3.10/lib-dynload', '', '/home/delucca/.pyenv/versions/3.10.14/envs/seismic-attributes-memory-profile/lib/python3.10/site-packages', '/home/delucca/src/unicamp/msc/seismic-attributes-memory-profile/tools/seismic', '/home/delucca/src/unicamp/msc/seismic-attributes-memory-profile/tools/traceq']


# Envelope

In [2]:
import dask
import time
import dask.array as da

from scipy import signal
from dask.diagnostics import ProgressBar, Profiler, ResourceProfiler
from dask.distributed import Client
from IPython.display import display
from bokeh.io import output_notebook
from seismic.data.synthetic import generate_and_save_synthetic_data
from seismic.data.loaders import load_segy

# Ensure Bokeh works properly in Jupyter
output_notebook()

# Disable GPU diagnostics in Dask
dask.config.set({"distributed.diagnostics.nvml": False})


# Minimal Envelope class
class Envelope:
    def __init__(self):
        pass

    def _lazy_transform_cpu(self, X):
        analytical_trace = da.map_blocks(signal.hilbert, X, dtype=X.dtype)
        return da.absolute(analytical_trace)

    def transform(self, X):
        return self._lazy_transform_cpu(X)


# Create a synthetic seismic experiment
synthetic_data_path = generate_and_save_synthetic_data(
    1200,
    600,
    200,
    output_dir="./",
)
synthetic_data = load_segy(synthetic_data_path)

# Instantiate the Envelope class
envelope_transform = Envelope()

Loading BokehJS ...

## Auto-chunking, single worker, high mem

In [18]:
# Start a Dask client
client = Client(n_workers=1, threads_per_worker=1, memory_limit='16GB')

# Create the experiment
print("Data shape: ", synthetic_data.shape)
X = da.from_array(synthetic_data, chunks='auto')
print("Chunks: ", X.chunks)
print("Number of chunks along each axis:", [len(c) for c in X.chunks])

# Apply the transformation (this is lazy)
envelope = envelope_transform.transform(X)

# Use Dask Profiler to monitor task execution and resource usage
profiler = Profiler()
resource_profiler = ResourceProfiler()

with ProgressBar(), profiler, resource_profiler:
    start_time = time.time()
    try:
        result = envelope.compute()
    finally:
        end_time = time.time()
        client.close()

# Execution Time
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

# Display the profiling visualizations in the notebook
profile_visualization = profiler.visualize()
resource_visualization = resource_profiler.visualize()

# Display using Bokeh inside the notebook
display(profile_visualization)
display(resource_visualization)

Data shape:  (1200, 200, 600)
Chunks:  ((409, 409, 382), (200,), (409, 191))
Number of chunks along each axis: [3, 1, 2]
Execution time: 4.55 seconds


Figure(id='8258', ...)

Figure(id='8632', ...)

## Single chunk, single worker, high mem

In [3]:
# Start a Dask client
client = Client(n_workers=1, threads_per_worker=1, memory_limit='16GB')

# Rechunk
print("Data shape: ", synthetic_data.shape)
X = da.from_array(synthetic_data, chunks=(1200, 200, 600))
print("Chunks: ", X.chunks)
print("Number of chunks along each axis:", [len(c) for c in X.chunks])

# Apply the transformation (this is lazy)
envelope = envelope_transform.transform(X)

# Use Dask Profiler to monitor task execution and resource usage
profiler = Profiler()
resource_profiler = ResourceProfiler()

with ProgressBar(), profiler, resource_profiler:
    start_time = time.time()
    try:
        result = envelope.compute()
    finally:
        end_time = time.time()
        client.close()

# Execution Time
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

# Display the profiling visualizations in the notebook
profile_visualization = profiler.visualize()
resource_visualization = resource_profiler.visualize()

# Display using Bokeh inside the notebook
display(profile_visualization)
display(resource_visualization)

Data shape:  (1200, 200, 600)
Chunks:  ((1200,), (200,), (600,))
Number of chunks along each axis: [1, 1, 1]
Execution time: 2.42 seconds


Figure(id='1003', ...)

Figure(id='1081', ...)

## Auto chunking, a few workers, small mem

In [6]:
client = Client(n_workers=3, threads_per_worker=1, memory_limit='3GB')

# Create the experiment
print("Data shape: ", synthetic_data.shape)
X = da.from_array(synthetic_data, chunks='auto')
print("Chunks: ", X.chunks)
print("Number of chunks along each axis:", [len(c) for c in X.chunks])

# Apply the transformation (this is lazy)
envelope = envelope_transform.transform(X)

# Use Dask Profiler to monitor task execution and resource usage
profiler = Profiler()
resource_profiler = ResourceProfiler()

with ProgressBar(), profiler, resource_profiler:
    start_time = time.time()
    try:
        result = envelope.compute()
    finally:
        end_time = time.time()
        client.close()

# Execution Time
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

# Display the profiling visualizations in the notebook
profile_visualization = profiler.visualize()
resource_visualization = resource_profiler.visualize()

# Display using Bokeh inside the notebook
display(profile_visualization)
display(resource_visualization)

Data shape:  (1200, 200, 600)
Chunks:  ((409, 409, 382), (200,), (409, 191))
Number of chunks along each axis: [3, 1, 2]
Execution time: 2.60 seconds


Figure(id='1525', ...)

Figure(id='1645', ...)

## Static chunking, a few workers, small mem

In [7]:
client = Client(n_workers=3, threads_per_worker=1, memory_limit='3GB')

# Create the experiment
print("Data shape: ", synthetic_data.shape)
X = da.from_array(synthetic_data, chunks=(400, 200, 600))
print("Chunks: ", X.chunks)
print("Number of chunks along each axis:", [len(c) for c in X.chunks])

# Apply the transformation (this is lazy)
envelope = envelope_transform.transform(X)

# Use Dask Profiler to monitor task execution and resource usage
profiler = Profiler()
resource_profiler = ResourceProfiler()

with ProgressBar(), profiler, resource_profiler:
    start_time = time.time()
    try:
        result = envelope.compute()
    finally:
        end_time = time.time()
        client.close()

# Execution Time
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

# Display the profiling visualizations in the notebook
profile_visualization = profiler.visualize()
resource_visualization = resource_profiler.visualize()

# Display using Bokeh inside the notebook
display(profile_visualization)
display(resource_visualization)

Data shape:  (1200, 200, 600)
Chunks:  ((400, 400, 400), (200,), (600,))
Number of chunks along each axis: [3, 1, 1]
Execution time: 1.42 seconds


Figure(id='1849', ...)

Figure(id='1990', ...)

# GST3D

In [8]:
import dask.array as da
import numpy as np
from scipy import ndimage as ndi


def trim_dask_array(in_data, kernel, hw=None, boundary='reflect'):
    # Compute half windows and assign to dict
    if hw is None:
        hw = tuple(np.array(kernel) // 2)
    axes = {0: hw[0], 1: hw[1], 2: hw[2]}

    return da.overlap.trim_internal(in_data, axes=axes, boundary=boundary)


# Minimal Gradient Structure Tensor for 3D Dip Calculation (CPU only)
class FirstDerivative:
    def __init__(self, axis=-1, preview=None):
        super().__init__()

        self._axis = axis
        self._preview = preview

    def __lazy_transform(self, X, xndi, xp):
        kernel = (3, 3, 3)
        axes = [ax for ax in range(X.ndim) if ax != self._axis]
        chunks_init = X.chunks

        result0 = X.map_blocks(xndi.correlate1d,
                               weights=xp.array([-0.5, 0.0, 0.5]),
                               axis=self._axis, dtype=X.dtype,
                               meta=xp.array((), dtype=X.dtype))
        result1 = result0.map_blocks(xndi.correlate1d,
                                     weights=xp.array([0.178947, 0.642105,
                                                       0.178947]),
                                     axis=axes[0], dtype=X.dtype,
                                     meta=xp.array((), dtype=X.dtype))
        result2 = result1.map_blocks(xndi.correlate1d,
                                     weights=xp.array([0.178947, 0.642105,
                                                       0.178947]),
                                     axis=axes[1], dtype=X.dtype,
                                     meta=xp.array((), dtype=X.dtype))

        return trim_dask_array(result2, kernel)

    def __transform(self, X, xndi, xp):
        axes = [ax for ax in range(X.ndim) if ax != self._axis]

        result0 = xndi.correlate1d(X, weights=xp.array([-0.5, 0.0, 0.5]),
                                   axis=self._axis)

        result1 = xndi.correlate1d(result0,
                                   weights=xp.array([0.178947, 0.642105,
                                                     0.178947]),
                                   axis=axes[0])

        result2 = xndi.correlate1d(result1,
                                   weights=xp.array([0.178947, 0.642105,
                                                     0.178947]),
                                   axis=axes[1])

        return result2

    def _lazy_transform_cpu(self, X):
        return self.__lazy_transform(X, ndi, np)

    def _transform_cpu(self, X):
        return self.__transform(X, ndi, np)


class GradientStructureTensor:
    def __init__(self, kernel, preview=None):
        super().__init__()

        self._kernel = kernel
        self._preview = preview

        self.__first_derivative_gi = FirstDerivative(axis=0)
        self.__first_derivative_gj = FirstDerivative(axis=1)
        self.__first_derivative_gk = FirstDerivative(axis=2)

    def _lazy_transform(self, gi, gj, gk, xndi):
        """Compute Inner Product of Gradients"""
        hw = tuple(np.array(self._kernel) // 2)

        gi2 = (gi * gi).map_overlap(xndi.uniform_filter, depth=hw,
                                    boundary='reflect',
                                    dtype=gi.dtype, size=self._kernel)
        gj2 = (gj * gj).map_overlap(xndi.uniform_filter, depth=hw,
                                    boundary='reflect',
                                    dtype=gj.dtype, size=self._kernel)
        gk2 = (gk * gk).map_overlap(xndi.uniform_filter, depth=hw,
                                    boundary='reflect',
                                    dtype=gk.dtype, size=self._kernel)
        gigj = (gi * gj).map_overlap(xndi.uniform_filter, depth=hw,
                                     boundary='reflect',
                                     dtype=gj.dtype, size=self._kernel)
        gigk = (gi * gk).map_overlap(xndi.uniform_filter, depth=hw,
                                     boundary='reflect',
                                     dtype=gk.dtype, size=self._kernel)
        gjgk = (gj * gk).map_overlap(xndi.uniform_filter, depth=hw,
                                     boundary='reflect',
                                     dtype=gj.dtype, size=self._kernel)

        return (gi2, gj2, gk2, gigj, gigk, gjgk)

    def _transform(self, gi, gj, gk, xndi):
        gi2 = xndi.uniform_filter(gi * gi, size=self._kernel)
        gj2 = xndi.uniform_filter(gj * gj, size=self._kernel)
        gk2 = xndi.uniform_filter(gk * gk, size=self._kernel)
        gigj = xndi.uniform_filter(gi * gj, size=self._kernel)
        gigk = xndi.uniform_filter(gi * gk, size=self._kernel)
        gjgk = xndi.uniform_filter(gj * gk, size=self._kernel)

        return (gi2, gj2, gk2, gigj, gigk, gjgk)

    def _lazy_transform_cpu(self, X):
        X_da = X
        chunks_init = X.chunks

        # Compute I, J, K gradients
        gi = self.__first_derivative_gi._lazy_transform_cpu(X)
        gj = self.__first_derivative_gj._lazy_transform_cpu(X)
        gk = self.__first_derivative_gk._lazy_transform_cpu(X)

        return self._lazy_transform(gi, gj, gk, ndi)

    def _transform_cpu(self, X):
        # Compute I, J, K gradients
        gi = self.__first_derivative_gi._transform_cpu(X)
        gj = self.__first_derivative_gj._transform_cpu(X)
        gk = self.__first_derivative_gk._transform_cpu(X)

        return self._transform(gi, gj, gk, ndi)


class GradientStructureTensor3DDip:
    def __init__(self, dip_factor=10, kernel=(3, 3, 3), preview=None):
        super().__init__()

        self._dip_factor = dip_factor
        self._kernel = kernel
        self._preview = preview

        self.__gst = GradientStructureTensor(kernel=kernel, preview=preview)

    def __operation(self, gi2, gj2, gk2, gigj, gigk, gjgk, axis, xp):
        """Function to compute 3D dip from GST"""
        if hasattr(xp, 'seterr'):
            xp.seterr(all='ignore')

        shape = gi2.shape

        gst = xp.array([[gi2, gigj, gigk],
                        [gigj, gj2, gjgk],
                        [gigk, gjgk, gk2]])

        gst = xp.moveaxis(gst, [0, 1], [-2, -1])
        gst = gst.reshape((-1, 3, 3))

        evals, evecs = xp.linalg.eigh(gst)
        ndx = evals.argsort()
        evecs = evecs[xp.arange(0, gst.shape[0], 1), :, ndx[:, 2]]

        norm_factor = xp.linalg.norm(evecs, axis=-1)
        evecs[:, 0] /= norm_factor
        evecs[:, 1] /= norm_factor
        evecs[:, 2] /= norm_factor

        evecs *= xp.sign(evecs[:, 2]).reshape(-1, 1)

        dip = xp.dot(evecs, xp.array([0, 0, 1], dtype=evecs.dtype))
        dip = xp.arccos(dip)
        dip = dip.reshape(shape)

        dip = xp.rad2deg(dip)

        return dip

    def _lazy_transform_cpu(self, X):
        # Compute Inner Product of Gradients and Dips
        gi2, gj2, gk2, gigj, gigk, gjgk = self.__gst._lazy_transform_cpu(X)

        result = da.map_blocks(self.__operation, gi2, gj2, gk2, gigj, gigk,
                               gjgk, axis=0, xp=np, dtype=X.dtype,
                               meta=np.array((), dtype=X.dtype))

        result[da.isnan(result)] = 0

        return result

    def _transform_cpu(self, X):
        # Compute Inner Product of Gradients and Dips
        gi2, gj2, gk2, gigj, gigk, gjgk = self.__gst._transform_cpu(X)

        result = self.__operation(gi2, gj2, gk2, gigj, gigk, gjgk, axis=0,
                                  xp=np)

        result[np.isnan(result)] = 0

        return result

    def transform(self, X):
        return self._lazy_transform_cpu(X)


# Create a synthetic seismic experiment
synthetic_data_path = generate_and_save_synthetic_data(
    600,
    200,
    100,
    output_dir="./",
)
synthetic_data = load_segy(synthetic_data_path)

# Instantiate the GST class
gst_transform = GradientStructureTensor3DDip()

## Auto-chunking, single worker, high mem

In [9]:
# Start a Dask client
client = Client(n_workers=1, threads_per_worker=1, memory_limit='16GB')

# Create the experiment
print("Data shape: ", synthetic_data.shape)
X = da.from_array(synthetic_data, chunks='auto')
print("Chunks: ", X.chunks)
print("Number of chunks along each axis:", [len(c) for c in X.chunks])

# Apply the transformation (this is lazy)
gst = gst_transform.transform(X)

# Use Dask Profiler to monitor task execution and resource usage
profiler = Profiler()
resource_profiler = ResourceProfiler()

with ProgressBar(), profiler, resource_profiler:
    start_time = time.time()
    try:
        result = gst.compute()
    finally:
        end_time = time.time()
        client.close()

# Execution Time
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

# Display the profiling visualizations in the notebook
profile_visualization = profiler.visualize()
resource_visualization = resource_profiler.visualize()

# Display using Bokeh inside the notebook
display(profile_visualization)
display(resource_visualization)

Data shape:  (600, 100, 200)
Chunks:  ((600,), (100,), (200,))
Number of chunks along each axis: [1, 1, 1]
Execution time: 6.82 seconds


Figure(id='2215', ...)

Figure(id='2377', ...)

## Single chunk, single worker, high mem

In [10]:
# Start a Dask client
client = Client(n_workers=1, threads_per_worker=1, memory_limit='16GB')

# Rechunk
print("Data shape: ", synthetic_data.shape)
X = da.from_array(synthetic_data, chunks=(600, 100, 200))
print("Chunks: ", X.chunks)
print("Number of chunks along each axis:", [len(c) for c in X.chunks])

# Apply the transformation (this is lazy)
gst = gst_transform.transform(X)

# Use Dask Profiler to monitor task execution and resource usage
profiler = Profiler()
resource_profiler = ResourceProfiler()

with ProgressBar(), profiler, resource_profiler:
    start_time = time.time()
    try:
        result = gst.compute()
    finally:
        end_time = time.time()
        client.close()

# Execution Time
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

# Display the profiling visualizations in the notebook
profile_visualization = profiler.visualize()
resource_visualization = resource_profiler.visualize()

# Display using Bokeh inside the notebook
display(profile_visualization)
display(resource_visualization)

Data shape:  (600, 100, 200)
Chunks:  ((600,), (100,), (200,))
Number of chunks along each axis: [1, 1, 1]
Execution time: 6.69 seconds


Figure(id='2623', ...)

Figure(id='2806', ...)

## Auto chunking, a few workers, small mem

In [11]:
# Start a Dask client
client = Client(n_workers=3, threads_per_worker=1, memory_limit='3GB')

# Rechunk
print("Data shape: ", synthetic_data.shape)
X = da.from_array(synthetic_data, chunks='auto')
print("Chunks: ", X.chunks)
print("Number of chunks along each axis:", [len(c) for c in X.chunks])

# Apply the transformation (this is lazy)
gst = gst_transform.transform(X)

# Use Dask Profiler to monitor task execution and resource usage
profiler = Profiler()
resource_profiler = ResourceProfiler()

with ProgressBar(), profiler, resource_profiler:
    start_time = time.time()
    try:
        result = gst.compute()
    finally:
        end_time = time.time()
        client.close()

# Execution Time
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

# Display the profiling visualizations in the notebook
profile_visualization = profiler.visualize()
resource_visualization = resource_profiler.visualize()

# Display using Bokeh inside the notebook
display(profile_visualization)
display(resource_visualization)

Data shape:  (600, 100, 200)
Chunks:  ((600,), (100,), (200,))
Number of chunks along each axis: [1, 1, 1]


2024-09-26 16:27:34,112 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.99 GiB -- Worker memory limit: 2.79 GiB
2024-09-26 16:27:35,016 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 2.24 GiB -- Worker memory limit: 2.79 GiB
2024-09-26 16:27:36,530 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:46593 (pid=2558) exceeded 95% memory budget. Restarting...
2024-09-26 16:27:36,567 - distributed.nanny - WARNING - Restarting worker
2024-09-26 16:27:38,389 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-

KilledWorker: Attempted to run task ('where-10533403875461cbfdbeaf5d956137ec', 0, 0, 0) on 3 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://127.0.0.1:43385. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.

## Static chunking, a few workers, small mem

In [12]:
# Start a Dask client
client = Client(n_workers=3, threads_per_worker=1, memory_limit='3GB')

# Rechunk
print("Data shape: ", synthetic_data.shape)
X = da.from_array(synthetic_data, chunks=(200, 100, 200))
print("Chunks: ", X.chunks)
print("Number of chunks along each axis:", [len(c) for c in X.chunks])

# Apply the transformation (this is lazy)
gst = gst_transform.transform(X)

# Use Dask Profiler to monitor task execution and resource usage
profiler = Profiler()
resource_profiler = ResourceProfiler()

with ProgressBar(), profiler, resource_profiler:
    start_time = time.time()
    try:
        result = gst.compute()
    finally:
        end_time = time.time()
        client.close()

# Execution Time
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

# Display the profiling visualizations in the notebook
profile_visualization = profiler.visualize()
resource_visualization = resource_profiler.visualize()

# Display using Bokeh inside the notebook
display(profile_visualization)
display(resource_visualization)

Data shape:  (600, 100, 200)
Chunks:  ((200, 200, 200), (100,), (200,))
Number of chunks along each axis: [3, 1, 1]
Execution time: 3.43 seconds


Figure(id='3073', ...)

Figure(id='3277', ...)